# This jupyter notebook loads de csv prices data and estimate the total price of the project 

In [201]:
%run functions.py

### Load the data and set the sema format

In [190]:
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

base_path = "/Users/Curi/DATAKNOWN/Prueba Técnica Cientifico de Datos/Prueba tecnica 1/Datos/"

X = load_csv_data(path = base_path + "X.csv", separation = ",", decimal = ".", reversed = True)
Y = load_csv_data(path = base_path + "Y.csv", separation = ";", decimal = ",", reversed = True)
Y["Date"] = pd.to_datetime(Y["Date"], format="%d/%m/%Y")
Y["Date"] = pd.to_datetime(Y["Date"], format="%Y-%m-%d")
Z = load_csv_data(path = base_path + "Z.csv", separation = ",", decimal = ".", reversed = False)

### Transform the Date into numerical values to perform the mathematical analysis easier

In [191]:
total_data = X["Date"].count()
max_date = X["Date"].iloc[-1]
min_date = X["Date"].iloc[0]
total_days_sampled = (max_date-min_date).days

X["Date_Num"] = total_days_sampled - (max_date-X["Date"]).dt.days
Y["Date_Num"] = total_days_sampled - (max_date-Y["Date"]).dt.days
Z["Date_Num"] = total_days_sampled - (max_date-Z["Date"]).dt.days

### Plot the raw data

In [192]:
fig, ax = plt.subplots(1,3,figsize=(30, 7))
ax[0].plot(X["Date_Num"],X["Price"])

ax[1].plot(Y["Date_Num"],Y["Price"])

ax[2].plot(Z["Date_Num"],Z["Price"])

for a in ax:
    a.tick_params(axis='both', which='major', labelsize=15)
    a.set_xlabel("Date", fontsize=25)
    
ax[0].set_ylabel("Price", fontsize=25)

ax[0].set_title("X", fontsize=25)
ax[1].set_title("Y", fontsize=25)
ax[2].set_title("Z", fontsize=25)

filename = 'raw_data.png'
plt.savefig(filename, dpi=300, bbox_inches='tight')
plt.close(fig)

### Fit the measured data to a specified function

In [202]:
from scipy.optimize import curve_fit

# X
poptX, pcovX = curve_fit(fit_X, X["Date_Num"], np.log(X["Price"]))
standar_deviationX = np.sqrt(np.diag(pcovX))
print("X fitted parameters=",poptX," with standar deviation",standar_deviationX)

# Y
poptY, pcovY = np.polyfit(Y["Date_Num"], Y["Price"], deg=1, cov=True)
pY = np.poly1d(poptY)
standar_deviationY = np.sqrt(np.diag(pcovY))
print("Y fited parameters=",poptY," with standar deviation",standar_deviationY)

# Z
poptZ, pcovZ = np.polyfit(Z["Date_Num"], Z["Price"], deg=1, cov=True)
pZ = np.poly1d(poptZ)
standar_deviationZ = np.sqrt(np.diag(pcovZ))
print("Z fited parameters=",poptZ," with standar deviation",standar_deviationZ)




X fitted parameters= [2.71093241e+00 1.51616116e-04]  with standar deviation [8.63049306e-03 1.13947409e-06]
Y fited parameters= [-8.46733305e-04  5.73690718e+02]  with standar deviation [1.19708425e-03 1.18388615e+01]
Z fited parameters= [5.11158077e-02 1.50784857e+03]  with standar deviation [4.25086078e-03 4.44358061e+01]


### Plot the fitted data

In [205]:
fig, ax = plt.subplots(1,3,figsize=(30, 7))

daysX = np.linspace(0,15000,8000)
daysYZ = np.linspace(7000,15000,8000)

ax[0].plot(X["Date_Num"],X["Price"])
ax[0].plot(daysX,np.exp(fit_X(daysX,poptX[0],poptX[1])))

ax[1].plot(Y["Date_Num"],Y["Price"])
ax[1].plot(daysYZ,pY(daysYZ))

ax[2].plot(Z["Date_Num"],Z["Price"])
ax[2].plot(daysYZ,pZ(daysYZ))

for a in ax:
    a.tick_params(axis='both', which='major', labelsize=15)

for k in range(3):
    ax[k].set_xlabel("Date", fontsize=25)
ax[0].set_ylabel("Price", fontsize=25)

ax[0].set_title("X", fontsize=25)
ax[1].set_title("Y", fontsize=25)
ax[2].set_title("Z", fontsize=25)

filename = 'fited_data.png'
plt.savefig(filename, dpi=300, bbox_inches='tight')
plt.close(fig)